In [2]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd

In [4]:

data_dir = '/global/cfs/cdirs/desi/users/chahah/sedflow/desiy1_bgs/'

# List all files and directories in the specified directory
files = os.listdir(data_dir)

# Print the list of files
for file in files:
    print(file)

BGS_ANY_full.vagc.82.hdf5
BGS_ANY_full.vagc.83.hdf5
BGS_ANY_full.vagc.68.hdf5
BGS_ANY_full.vagc.73.hdf5
BGS_ANY_full.vagc.53.hdf5
BGS_ANY_full.vagc.0.hdf5
BGS_ANY_full.vagc.98.hdf5
BGS_ANY_full.vagc.85.hdf5
BGS_ANY_full.vagc.41.hdf5
BGS_ANY_full.vagc.93.hdf5
BGS_ANY_full.vagc.67.hdf5
BGS_ANY_full.vagc.45.hdf5
BGS_ANY_full.vagc.48.hdf5
BGS_ANY_full.vagc.88.hdf5
BGS_ANY_full.vagc.61.hdf5
BGS_ANY_full.vagc.44.hdf5
BGS_ANY_full.vagc.36.hdf5
BGS_ANY_full.vagc.66.hdf5
BGS_ANY_full.fluxes.hdf5
BGS_ANY_full.vagc.50.hdf5
BGS_ANY_full.vagc.71.hdf5
BGS_ANY_full.vagc.58.hdf5
BGS_ANY_full.vagc.105.hdf5
BGS_ANY_full.vagc.104.hdf5
BGS_ANY_full.vagc.8.hdf5
BGS_ANY_full.vagc.87.hdf5
BGS_ANY_full.vagc.39.hdf5
BGS_ANY_full.vagc.78.hdf5
BGS_ANY_full.vagc.108.hdf5
BGS_ANY_full.vagc.76.hdf5
BGS_ANY_full.vagc.86.hdf5
BGS_ANY_full.vagc.55.hdf5
BGS_ANY_full.vagc.103.hdf5
BGS_ANY_full.vagc.101.hdf5
BGS_ANY_full.vagc.59.hdf5
BGS_ANY_full.vagc.100.hdf5
BGS_ANY_full.vagc.99.hdf5
BGS_ANY_full.vagc.46.hdf5
BGS_ANY_f

In [6]:
t1 = Table.read(data_dir + 'BGS_ANY_full.vagc.83.hdf5') 
t1

TARGETID,Z_not4clus,Z_not4clus.mask,ZSUCCESS,ZSUCCESS.mask,FLUX_G_CORR,FLUX_R_CORR,FLUX_Z_CORR,FLUX_W1_CORR,FLUX_W2_CORR,FLUX_SIG_G,FLUX_SIG_R,FLUX_SIG_Z,FLUX_SIG_W1,FLUX_SIG_W2,SEDFLOW_SAMPLES,SEDFLOW_MAP,SEDFLOW_LOGMSTAR_SAMPLES,SEDFLOW_LOGMSTAR_MAP
int64,float64,bool,bool,bool,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,"float64[100,15]",float64[15],float64[100],float64
39633063465976717,0.1984321455931371,False,True,False,4.0869265,8.941882,15.19743,14.470984,11.8612175,0.06631819,0.12453196,0.22856705,0.5026216,0.9698879,10.00761604309082 .. 0.3022882640361786,10.120773315429688 .. 0.4905169904232025,9.831948144010942 .. 10.265313796445604,9.902524917942758
39633063465976775,0.07739264262557048,False,True,False,28.644543,65.05548,110.621346,78.36352,47.595844,0.0941359,0.17433782,0.22364667,0.6509403,1.084444,10.3016357421875 .. 0.6046794652938843,10.095832824707031 .. 0.447418749332428,10.073417838337866 .. 10.110198468252259,9.866170018548555
39633063465976835,0.4224541460834488,False,True,False,3.7730072,10.41048,17.686543,21.95593,11.882837,0.06851298,0.12872428,0.17569321,0.5236661,0.97102827,10.98884391784668 .. 0.23502299189567566,10.824398040771484 .. 0.4599613547325134,10.75931323790912 .. 10.806520280683593,10.60658494401825
39633063465976851,0.1988486466995249,False,True,False,13.923654,27.96904,45.1318,45.26602,36.140762,0.079065554,0.14567786,0.1890766,0.58015245,1.0523483,10.446907043457031 .. 0.37849748134613037,10.539756774902344 .. 0.3534317910671234,10.251751483533104 .. 10.271690392119185,10.333368343450003
39633063465976929,0.11272636245329602,False,True,False,32.262615,76.005646,132.64453,112.87609,70.987366,0.08572374,0.17226325,0.20863734,0.6902517,1.0981777,10.77932357788086 .. 0.2845594584941864,10.568489074707031 .. 0.2510432302951813,10.538865712987155 .. 10.163046018292173,10.342730885010392
39633063465976982,0.43166843458918663,False,True,False,2.9264965,16.035316,40.397755,74.12311,47.49677,0.06792767,0.1522323,0.23534743,0.6646752,1.125637,11.602624893188477 .. 0.5453040599822998,11.502645492553711 .. 0.27889829874038696,11.361156461293168 .. 11.369292928261572,11.271730836883995
39633063465977053,0.19881416791443085,False,True,False,3.421629,9.766458,18.324303,19.75731,13.340188,0.04771845,0.097762845,0.13655895,0.47931257,0.90949756,10.315646171569824 .. 0.30980581045150757,10.25619888305664 .. 0.4761716425418854,10.096653746497779 .. 9.952406024423773,10.032473317591176
39633063465977055,0.32761482004445586,False,True,False,3.2650013,11.406072,20.203323,23.385798,15.512587,0.04629294,0.09595724,0.14920214,0.4994048,0.92161626,10.392745971679688 .. 0.46625539660453796,10.686664581298828 .. 0.2595478892326355,10.21391794466259 .. 10.480104559050684,10.467570677193414
39633063465977143,0.06768815094235989,False,True,False,8.228089,10.443602,11.360152,7.1603303,3.7709343,0.05904536,0.106773585,0.17546718,0.45923322,0.90138984,8.910362243652344 .. 0.5420528054237366,8.742534637451172 .. 0.5003098845481873,8.710901533736397 .. 8.570407117987237,8.531179568876437
